In [1]:
import json
import re

In [19]:
file = '/data/lyz/math_dpo/output/gsmhard_gen_naive.json'
with open(file) as f:
    datas = json.load(f)

In [5]:
def extract_numerical_answer(solutions,target):
    answers = []
    for i,solution in enumerate(solutions):
        # 使用正则表达式提取 "So, the numerical answer is:" 后的数字
        match = re.search(r"So, the numerical answer is:(.*)", solution, re.IGNORECASE | re.DOTALL)

        if match:
            # 提取出这部分文本内容
            answer_line = match.group(1).strip()
        else:
            answer_line = solution.split('\n')[-1]
            # 从中提取数字（包括整数和小数）
        num_match = re.findall(r"[-+]?\d+(?:,\d{3})*(?:\.\d+)?", answer_line)

        if num_match:
            flag = 0
            for num in num_match:
                final_answer_str = num.replace(',', '').replace(' ', '')
                try:
                    # 转换为浮点数或整数
                    final_answer = float(final_answer_str) if '.' in final_answer_str else int(final_answer_str)
                    if final_answer == target:
                        answers.append(final_answer)
                        flag = 1 
                        break
                except ValueError:
                    print('error')
                    answers.append(None)  # 如果转换失败设为 None
            if flag == 0:
                final_answer = float(final_answer_str) if '.' in final_answer_str else int(final_answer_str)
                answers.append(final_answer)
        else:
            answers.append(None)
    return answers

def eval_answer(answers,target):
    labels = []
    for ans in answers:
        if not ans:
            labels.append(None)
        else:
            labels.append(float(ans) == target)
    return labels

In [ ]:
datas[0]

In [20]:
cnt = 0
datas_select = []
for ind,data in enumerate(datas):
    solutions = data['solutions']
    target = data['final_answer']
    answers = extract_numerical_answer(solutions,target)
    is_answer = eval_answer(answers,target)
    
    data['answers_extract'] = answers
    data['is_answer'] = is_answer
    if len(answers) != len(solutions):
        print('*'*10 + 'error!'+'*'*10)
    right = 0
    wrong = 0
    for i,l in enumerate(is_answer):
        if l == False:
            wrong = 1
            # print(ind)
            # print(target)
            # print(solutions[i])
            # break
        elif l:
            right = 1 
    if right and wrong:
        cnt += 1 
        print(ind)
        print(answers)
        print(is_answer)
        datas_select.append(data)
print(cnt)
print(cnt/len(datas))

4
[21459059, 21459061, 21459060, 21459061, 21459061, 21459056, 21459056, 21459061, 21459056, 6]
[False, True, False, True, True, False, False, True, False, False]
6
[40414.95, 74363508, 64663920, 1859087.7, 40415, 5, 1859087.7, 74363508, 64663920, 161664800]
[False, True, False, False, False, False, False, True, False, False]
8
[17413874, 17413874, 17414074, 17413594, 17413633, 17413874, 174140, 17413584, 204142506, 17414074]
[False, False, True, False, False, False, False, False, False, True]
9
[None, None, 36, 1, 1, 2.65e-05, 6, 1, 1, 1]
[None, None, False, True, True, False, False, True, True, True]
10
[125, None, 125, 96, 125, 7, 125, 125, 125, 125]
[True, None, True, False, True, False, True, True, True, True]
11
[8404732, 6277302500, 1255494800, 8404732, None, 8404732, 72708460, 6277334000, 8404732, 8404732]
[False, False, False, False, None, False, False, True, False, False]
17
[25124312, 25124292, 25124292, 8375242, 8375242, 25124292, 25124292, 25124292, 25124292, 25124992]
[Fa

In [17]:
datas[0]['solutions'][1]

'To find out how many clips Natalia sold in May, we need to calculate half of the amount she sold in April. Since she sold 48 clips in April:\n\n[Step 1]: Clips sold in May = 48 / 2\n\n[Step 2]: So, the numerical answer is: 24'

In [21]:
file = '/data/lyz/math_dpo/datas/gsmhard_select_2.json'
with open(file,'w')as f:
    json.dump(datas_select,f,indent = 4)

In [ ]:
datas[30]

In [ ]:
for ind,data in enumerate(datas_select):
    target = data['final_answer']
    solutions = data['solutions']
    answers = data['answers_extract']
    ans_set = set()
    data['reject'] = []
    data['chosen'] = []
    flag = 0

    for i,ans in enumerate(answers):
        if ans != target and ans not in ans_set:
            data['reject'].append(solutions[i])
            ans_set.add(ans)
        elif ans == target and len(data['chosen']) == 0:
            data['chosen'].append(solutions[i])
    # print(ind)
    # print(data)

In [ ]:
file = '/data/lyz/math_dpo/datas/sol2sol/gsm8k_sol2sol_naive.json'
with open(file,'w') as f:
    json.dump(datas_select,f,indent = 4)

In [ ]:
def extract_numerical_answer_hq(solutions,target):
    answers = []
    for i,solution in enumerate(solutions):
        # 使用正则表达式提取 "So, the numerical answer is:" 后的数字
        match = re.search(r"So, the numerical answer is:(.*)", solution, re.IGNORECASE | re.DOTALL)

        if match:
            # 提取出这部分文本内容
            answer_line = match.group(1).strip()
            num_match = re.findall(r"[-+]?\d+(?:,\d{3})*(?:\.\d+)?", answer_line)
            if num_match:
                
                final_answer_str = num_match[0].replace(',', '').replace(' ', '')
                try:
                    # 转换为浮点数或整数
                    final_answer = float(final_answer_str) if '.' in final_answer_str else int(final_answer_str)
                    answers.append(final_answer)
                except ValueError:
                    print('error')
                    answers.append(None)  # 如果转换失败设为 None

        else:
            answers.append(None)
    return answers

In [ ]:
file1 = '/data/lyz/math_dpo/datas/gsm8k_select.json'
with open(file) as f:
    datas_select = json.load(f)


In [ ]:
len(datas_select)

In [ ]:
data_right = []
for data in datas_select:
    solutions = data['solutions']
    is_answer = data['is_answer']
    right_answer = []
    for i,a in enumerate(is_answer):
        if a:
            right_answer.append(solutions[a])
    data['right_answer'] = right_answer

In [ ]:
import re
import random

def split_and_sample_steps(text):
    # 检查是否存在 [Step ] 或 [step ] 标签
    if re.search(r"\[Step\s*\d+\]", text, re.IGNORECASE):
        # 使用 [Step ] 或 [step ] 标签进行分割
        steps = re.split(r"\[Step\s*\d+\]:?", text, flags=re.IGNORECASE)
    else:
        # 如果没有找到 [Step ] 标签，用换行符 \n\n 或 \n 分割
        steps = re.split(r"\n\n|\n", text)
    
    # 移除空白步骤并去除每个步骤的前后空格
    steps = [step.strip() for step in steps if step.strip()]
    
    # 获取实际步骤数
    step_count = len(steps)
    
    # 随机生成一个 k 值，在 1 到 step_count 范围内
    k = random.randint(0, step_count-1)
    
    # 返回前 k 个步骤
    sampled_steps = steps[:k]
    restored_steps = [f"[Step {i + 1}]:\n{step}" for i, step in enumerate(sampled_steps)]
    
    # 将恢复后的步骤组合成完整文本
    restored_text = "\n\n".join(restored_steps)
    return restored_text


In [ ]:
for i,data in enumerate(datas_select):
    right = data['right_answer']
    print(i)
    data['sample_step'] = []
    for r in right:
        text  = '[Step 1]: \n' + r
        sample_steps = split_and_sample_steps(text)
        data['sample_step'].append(sample_steps)
        

In [ ]:
file = '/data/lyz/math_dpo/datas/step2step/gsm8k_step2step_sample.json'
with open(file,'w')as f:
    json.dump(datas_select,f,indent = 4)

In [ ]:
len(datas_select)

In [ ]:
file ='/data/lyz/math_dpo/output/gsm8k_test_gen_naive.json'
with open(file)as f:
    datas = json.load(f)

In [ ]:
cnt = 0
for ind,data in enumerate(datas):
    gen = data['solutions']
    target = data['final_answer']
    answers = extract_numerical_answer(gen,target)
    is_answer = eval_answer(answers,target)
    
    solutions = data['solutions']
    print(ind)
    print(solutions[0])


In [ ]:
print(cnt/len(datas))
print(len(datas))